In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.metrics import log_loss
from sklearn.calibration import CalibratedClassifierCV
from sklearn.model_selection import KFold

plt.style.use('ggplot')
%matplotlib inline

x_train = pd.read_csv('data/x_train_norm.csv', sep=';', na_values='?')
x_test = pd.read_csv('data/x_test_norm.csv', sep=';', na_values='?')
y_train = pd.read_csv('data/y_train.csv', sep=';', header=None, na_values='?')[0]

def scorer_logloss(estimator, X, y):
    return log_loss(y, estimator.predict_proba(X)) # аналогично этому: estimator.predict_proba(X)[:,1]

def revert_scorer_logloss(estimator, X, y):
    return -1 * scorer_logloss(estimator, X, y)

In [5]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [1, 2]
             }


DTC = DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight='balanced',max_depth = None)

ABC = AdaBoostClassifier(base_estimator = DTC)

# run grid search
grid_search_ABC = GridSearchCV(ABC, param_grid=param_grid,                         
                        scoring=revert_scorer_logloss,
                        n_jobs=4,
                        iid=False,
                        cv=3)
grid_search_ABC.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            presort=False, random_state=11, splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [1, 2], 'base_estimator__criterion': ['gini', 'entropy'], 'base_estimator__splitter': ['best', 'random']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=<function revert_scorer_logloss at 0x1112b5c80>, verbose=0)

In [7]:
pd.DataFrame(grid_search_ABC.cv_results_)

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_base_estimator__criterion,param_base_estimator__splitter,param_n_estimators,params,rank_test_score,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,0.173365,0.025995,-8.148977,-0.016885,gini,best,1,"{u'n_estimators': 1, u'base_estimator__criteri...",6,-8.142502,-0.017418,-8.088714,-0.017481,-8.215713,-0.015756,0.058271,0.020648,0.052049,0.000799
1,0.231922,0.024139,-4.626738,-0.028653,gini,best,2,"{u'n_estimators': 2, u'base_estimator__criteri...",4,-4.561386,-0.029474,-4.570390,-0.029016,-4.748438,-0.027470,0.030758,0.007492,0.086133,0.000857
2,0.133326,0.013008,-8.260953,-0.016885,gini,random,1,"{u'n_estimators': 1, u'base_estimator__criteri...",8,-8.380067,-0.017418,-8.174485,-0.017481,-8.228309,-0.015756,0.012465,0.001672,0.087045,0.000799
3,0.180477,0.022161,-4.500281,-0.028653,gini,random,2,"{u'n_estimators': 2, u'base_estimator__criteri...",1,-4.488087,-0.029474,-4.505807,-0.029016,-4.506948,-0.027470,0.011043,0.002413,0.008635,0.000857
4,0.174202,0.010607,-8.149043,-0.016885,entropy,best,1,"{u'n_estimators': 1, u'base_estimator__criteri...",7,-8.220298,-0.017418,-8.142094,-0.017481,-8.084737,-0.015756,0.035057,0.000543,0.055560,0.000799
5,0.331720,0.025015,-4.609230,-0.028653,entropy,best,2,"{u'n_estimators': 2, u'base_estimator__criteri...",3,-4.730636,-0.029474,-4.581328,-0.029016,-4.515726,-0.027470,0.025714,0.007842,0.089928,0.000857
6,0.125649,0.021476,-8.067120,-0.016885,entropy,random,1,"{u'n_estimators': 1, u'base_estimator__criteri...",5,-8.269887,-0.017418,-7.969640,-0.017481,-7.961833,-0.015756,0.018676,0.002897,0.143414,0.000799
7,0.178435,0.017288,-4.547012,-0.028653,entropy,random,2,"{u'n_estimators': 2, u'base_estimator__criteri...",2,-4.487031,-0.029474,-4.518054,-0.029016,-4.635953,-0.027470,0.021736,0.003446,0.064153,0.000857


In [8]:
param_grid = {"base_estimator__criterion": ["gini", "entropy"],
              "base_estimator__splitter":  ["best", "random"],
              "n_estimators": range(10,101,10) #[1, 2]
             }

grid_search = GridSearchCV(
    AdaBoostClassifier(
        base_estimator = DecisionTreeClassifier(

           min_samples_split=500,
           min_samples_leaf=50,
           max_depth=8,
           max_features='sqrt',

            random_state = 11,
            #max_features = "auto",
            class_weight='balanced',
            #max_depth = None
            )),
    param_grid=param_grid,                         
    scoring=revert_scorer_logloss,
    n_jobs=4,
    iid=False,
    cv=3)

grid_search.fit(x_train, y_train)

GridSearchCV(cv=3, error_score='raise',
       estimator=AdaBoostClassifier(algorithm='SAMME.R',
          base_estimator=DecisionTreeClassifier(class_weight='balanced', criterion='gini', max_depth=8,
            max_features='sqrt', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=50,
            min_samples_split=500, min_weight_fraction_leaf=0.0,
            presort=False, random_state=11, splitter='best'),
          learning_rate=1.0, n_estimators=50, random_state=None),
       fit_params={}, iid=False, n_jobs=4,
       param_grid={'n_estimators': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100], 'base_estimator__criterion': ['gini', 'entropy'], 'base_estimator__splitter': ['best', 'random']},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=<function revert_scorer_logloss at 0x1112b5c80>, verbose=0)

In [9]:
grid_search.best_params_, -grid_search.best_score_

({'base_estimator__criterion': 'gini',
  'base_estimator__splitter': 'best',
  'n_estimators': 10},
 0.61858119034551518)

In [15]:
param_grid = {'base_estimator__max_depth':range(5,16,2),
              'base_estimator__min_samples_split':range(200,1001,200),
              'n_estimators': range(10,101,10)}

grid_search = GridSearchCV(
    AdaBoostClassifier(
        base_estimator = DecisionTreeClassifier(
            criterion='gini',
            splitter='best',
            min_samples_split=500,
            min_samples_leaf=50,
            max_depth=8,
            max_features='sqrt',
            random_state = 11,
            class_weight='balanced'),
        n_estimators=10),
    param_grid=param_grid,  
    scoring=revert_scorer_logloss,
    n_jobs=4,
    iid=False,
    cv=3)

grid_search.fit(x_train, y_train)
grid_search.best_params_, -grid_search.best_score_

({'base_estimator__max_depth': 15,
  'base_estimator__min_samples_split': 200,
  'n_estimators': 10},
 0.59645315843019786)

In [22]:
classifier = GradientBoostingClassifier(learning_rate=0.1,
                                        min_samples_leaf=50,
                                        n_estimators=60,
                                        max_features='sqrt',
                                        subsample=0.8,
                                        random_state=11)

ada_classifier = AdaBoostClassifier(base_estimator = classifier,
                                    n_estimators=2,
                                    learning_rate=0.1)

print cross_val_score(ada_classifier, x_train,y_train, scoring=revert_scorer_logloss)
print cross_val_score(classifier, x_train,y_train, scoring=revert_scorer_logloss)

[-0.38583237 -0.38227455 -0.38755924]
[-0.3849294  -0.38150752 -0.38672031]
